# Notebook for Calculating 

Basic diagnostic on precipitation - enable comparison between two simulations

- Plot monthly mean precipitation and precipitation difference between simulation 1 & 2
- Compute seasonal cycle of precipitation over selected area


# Running the notebook
Execute the cells 1 by 1 by taping `ctrl + enter` or `shift + enter`

__Make sure to upload 2 file before continuing the rest of the notebook__

If you want to change the longitude and latitude bounds look in the data extraction section.

# Imports

In [ ]:
# Download new color palette
#Crameri : 
#palettable.scientific.sequential / palettable.scientific.diverging
#from palettable.scientific.sequential import Oslo_10
#cmap=Oslo_10.mpl_colormap
#https://jiffyclub.github.io/palettable/#documentation
!pip install palettable
from palettable.scientific.sequential import Davos_10_r
from palettable.scientific.diverging import Vik_10


In [ ]:
# File upload widegt
from ipywidgets import FileUpload
from io import BytesIO

# Data manipulation and storage
import xarray as xr
import numpy as np

# Holoviews graphing
import hvplot.xarray
import holoviews as hv
from holoviews import opts

In [ ]:
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
import cartopy.feature as cf

gv.extension('bokeh', 'matplotlib')

# Data Upload

In [ ]:
# Create file upload
# To load several files press shift with selecting - 
#all the files have to be selected at the same time.

file_upload = FileUpload(multiple=True)
file_upload

In [ ]:
bytes_data = BytesIO(file_upload.data[0])
ds = xr.open_dataset(bytes_data)

bytes_data = BytesIO(file_upload.data[1])
ds1 = xr.open_dataset(bytes_data)

In [ ]:
#Scale precipitation to convert in mm/day
ds['precip'] = ds['precip']*86400
ds1['precip'] = ds1['precip']*86400

In [ ]:
ds[['precip', 'phis']].precip.hvplot(title="Dataset 1",
                                     width=300,
                                     height=300) + ds1[['precip', 'phis']].precip.hvplot(title="Dataset 2",
                                                                                                   width=300,
                                                                                                   height=300)

In [ ]:
alti = ds['phis']/9.81
alti.hvplot()


# Plot Maps

In [ ]:
# Plot monthly precipitations
topo = gv.Dataset(alti).to(gv.Image, ['lon','lat'],'phis')

opts.defaults(opts.Image(cmap=Davos_10_r.mpl_colormap,color_levels=10,clim=(0,20),frame_width=400,frame_height=200))

ensemble = gv.Dataset(ds).to(gv.Image, ['lon', 'lat'], 'precip')
ensemble1 = gv.Dataset(ds1).to(gv.Image, ['lon', 'lat'], 'precip')
(ensemble.opts(title="Precipitation (mm/day)",projection=crs.Robinson())*
 gv.operation.contours(topo,levels=[10]).opts(show_legend=False) + 
 ensemble1.opts(colorbar=True,title="Precipitation (mm/day)",projection=crs.Robinson())*
 gv.operation.contours(topo,levels=[10]).opts(show_legend=False))


In [ ]:
# Compute difference between two simulations
difference = (ds.precip - ds1.precip)
diff_zone = difference.sel(lat=slice(40, -40), lon=slice(-180, 180))

In [ ]:
# Plot differences
alti_zoom = alti.sel(lat=slice(40,-40),lon=slice(-180,180))
topo_zoom = gv.Dataset(alti_zoom).to(gv.Image, ['lon','lat'],'phis')

(gv.Dataset(diff_zone).to(gv.Image, ['lon', 'lat'], 'precip').opts(width=600,height=300,
                                                                  cmap=Vik_10.mpl_colormap,color_levels=20,clim=(-15,15),
                                                                  xlabel='Longitude',ylabel='Latitude',
                                                                  title='Precipitation anomaly (mm/day)')*
 gv.operation.contours(topo_zoom,levels=[10]).opts(show_legend=False))

# Data extraction 

Here we take only the variables we need from the dataset

In [ ]:
ds_precip = ds[['precip', 'phis']]
ds1_precip = ds[['precip', 'phis']]

# Convert to mm/day
ds_precip['precip'] =  ds_precip['precip'] *86400
ds1_precip['precip'] =  ds1_precip['precip'] *86400

In [ ]:
# Select the desired zone
# /!\ Slice works based on the ordering in coordinates, here lat goes from +90 -> -90 and lon from -180 -> +180
ds_precip_zone = ds_precip.sel(lat=slice(35, 0), lon=slice(65, 85))
# Calculate the altitude from the phi parameter
ds_precip_zone['alti'] = ds_precip_zone['phis'] / 9.81

# Analysis

In [ ]:
ds_precip_zone.precip.hvplot.image(label='precip')

In [ ]:
ds_precip_zone.alti.hvplot.image(label='alti')

In [ ]:
# Extract values above a threshold
threshold = 1000
upper = ds_precip_zone.where(ds_precip_zone.alti >= threshold)

# Extract values below threshold but above sea level
lower = ds_precip_zone.where((ds_precip_zone.alti > 0) & (ds_precip_zone.alti < threshold))

In [ ]:
upper.hvplot.image()

In [ ]:
lower.hvplot.image()

# Calculate the mean value for each time step

In [ ]:
upper.precip.mean(dim=['lat', 'lon']).plot.line();

In [ ]:
lower.precip.mean(dim=['lat', 'lon']).plot.line();

# Calculate the sum over all locations at each time step

In [ ]:
lower.precip.sum(dim=['lat', 'lon']).plot.line();

In [ ]:
lower.precip.sum(dim=['lat', 'lon']).plot() + upper.precip.sum(dim=['lat', 'lon']).plot();

# Calculate the cumlative sum for each time step

In [ ]:
upper.precip.sum(dim=['lat', 'lon']).cumsum().plot.line();

In [ ]:
lower.precip.sum(dim=['lat', 'lon']).cumsum().plot.line();